# Importações Necessárias

In [1]:
import numpy as np;
import pandas as pd;

# Parâmetros de Entrada

In [2]:
baseDados = "BPressureNishiBook.dat";
baseRegras = "BPressureNishiBook.txt";

# Pré-Processamento Base Transacional

In [98]:
# lendo arquivo
mt = pd.read_table(baseDados, delim_whitespace=True, dtype="str", header=None);
# pre-processando
mtBinaria = pd.get_dummies(mt);
dados = mtBinaria.astype('int').to_numpy();
print(dados);

[[1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0]
 [1 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 1]
 [0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 0 1]
 [0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 0]
 [0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 1 0]
 [0 1 0 1 0 0 0 1 0 0 0 1 0 0 1 1 0 0]
 [0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1]
 [1 0 0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 1]
 [0 1 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0]
 [1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1]
 [0 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0]
 [0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 0]
 [0 0 1 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0]
 [1 0 0 0 0 1 1 0 0 0 1 0 1 0 0 1 0 0]
 [0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 0 1 0]]


# Pré-processamento RAs

In [82]:
regras = pd.read_table(baseRegras, sep="#", names=("AR", "sup", "cnf"));
regras["sup"] = regras["sup"].str.replace("SUP:", "");
regras["cnf"] = regras["cnf"].str.replace("CONF:", "");

regras["itemset"] = regras["AR"].str.replace("==>", " ").str.split();

regras["antc"], regras["cons"] =  regras["AR"].str.split("==>").str;
regras["cons"] = regras["cons"].str.split();
regras["antc"] = regras["antc"].str.split();



print(regras);



                       AR  sup                   cnf                itemset  \
0                6 ==> 1    5     0.5555555555555556                 [6, 1]   
1                1 ==> 6    5                    1.0                 [1, 6]   
2                4 ==> 2    3                    1.0                 [4, 2]   
3                2 ==> 4    3                    0.5                 [2, 4]   
4                5 ==> 2    3                    1.0                 [5, 2]   
...                   ...  ...                   ...                    ...   
1369  15 ==> 3 6 9 12 16    1     0.3333333333333333  [15, 3, 6, 9, 12, 16]   
1370  12 ==> 3 6 9 15 16    1                    0.1  [12, 3, 6, 9, 15, 16]   
1371  9 ==> 3 6 12 15 16    1    0.16666666666666666  [9, 3, 6, 12, 15, 16]   
1372  6 ==> 3 9 12 15 16    1     0.1111111111111111  [6, 3, 9, 12, 15, 16]   
1373  3 ==> 6 9 12 15 16    1                   0.25  [3, 6, 9, 12, 15, 16]   

      antc                cons  
0      [6]        

# Calculo de Suporte

In [116]:
def calcSupp(antecedente, consequente):
    itemset = list(antecedente) + list(consequente);
    suporte = dados[:,itemset[0]-1]
    
    for i in itemset[1:]:
        suporte *= dados[:,i-1];
        
    return np.sum(suporte, axis=0)/ dados.shape[0] ;


print(calcSupp([12, 17, 14], [2]))



#print(np.sum(dados[:,0]*dados[:,5]/dados.shape[0]))


        
    



0.13333333333333333
